<a href="https://colab.research.google.com/github/Southsidegbemijo/DistilBERT-for-Healthcare-Implementation-science-model-PRISM-/blob/main/Text_Classifier_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate accelerate -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 20.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from datasets import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## We mount Google Drive and Load the Cleaned Spreadsheet



In [ ]:
%cd /content/drive/MyDrive/NLP Project with Drs. Jesso & Maw/NLP

/content/drive/MyDrive/NLP Project with Drs. Jesso & Maw/NLP


In [ ]:
sheet = pd.read_excel('Sheets1&2Merged.xlsx')
sheet.head()

,Participant,full_quote,sub_quote,question_from_interviewer,multi_level_org_char,multi_level_org_perspect,impl_sust_infra,Provider characteristcs,Health System characteristics (academics; bed capacity also),Imaging modalities in general,Value equation: quality/cost/efficiency/patient satisfaction (Hosp leaders-background for implementation),Clinical utility & efficiency-Provider perspective,Patient/Physican interaction in LUS,"Workflow (with subcoding [bolded]: access equipment, order vs encounter-based; uploading & saving images; type of equipment. End user need to acquire and use",training,credientialing /quality assurance infrastructure,Finanicial Impact,Clinical utility & efficiency-Provider perspective,Value equation: quality/cost/efficiency/patient satisfaction (Hosp leaders) High-value care (providers)
0,G7,"No, I don't. I think it was more to drive just...",Desire for Quality of care promted policy deve...,Got it. They saw this need and they thought yo...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,G7,I thought it was only gonna take us six months...,need for POCUS Chair: think you have to ident...,Any other thoughts about—how long is this gonn...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,G7,I don't have an assessment of that. Everybody ...,Cost of quality assurance team: Everybody want...,Is the hospital sort of perspective like that'...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,G4,I think it's variable and that's why it's so h...,Different decision-makers rs depending on the ...,"Yeah. Well, that's all music to my ears. I thi...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,G4,When I think from my perspective at the end of...,Opportunities to improve quality of care: When...,What is their role in our management in our ho...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
sheet.shape

(134, 19)

We will need to convert all the sparse columns with flagged 0 and 1 values into  a single column for easier training

In [ ]:

full_data = sheet.copy()

full_data['target'] = 'Unknown'
for col in full_data.columns[4:]:
  for index,row in full_data.iterrows():
    if row[col] == 1:
      full_data.loc[index,'target'] = col

labels = full_data.target.unique().tolist()

label2id = {i:labels.index(i) for i in labels}
id2label = {v:k for k,v in label2id.items()}

In [ ]:
labels

['Value equation: quality/cost/efficiency/patient satisfaction (Hosp leaders-background for implementation)',
 'credientialing /quality assurance infrastructure',
 'Finanicial Impact',
 'Health System characteristics (academics; bed capacity also)',
 'Clinical utility & efficiency-Provider perspective',
 'Workflow (with subcoding [bolded]: access equipment, order vs encounter-based; uploading & saving images; type of equipment.  End user need to acquire and use',
 'Provider characteristcs',
 'training',
 'Patient/Physican interaction in LUS',
 'Imaging modalities in general',
 'Clinical utility &  efficiency-Provider perspective',
 'Value equation: quality/cost/efficiency/patient satisfaction (Hosp leaders) High-value care (providers)']

the code above is to add a target column that further converts the sparse fetures into a single target column

## Fine-Tuning all the texts with the Transformers Library

Here, I'm using just the full quotes for the finetuning, to see how it performs

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments, Trainer,DataCollatorWithPadding,DataCollator
from sklearn.model_selection import train_test_split


MAX_LENGTH = 512

device = 'cuda' if torch.cuda.is_available() else 'cpu'


train_data, eval_data = train_test_split(full_data, test_size=0.2, random_state=42, stratify=full_data.target)

# Create datasets from the split data
train_dataset = Dataset.from_dict({
    'text': train_data.full_quote.values,
    'label': [labels.index(label) for label in train_data.target]
})

eval_dataset = Dataset.from_dict({
    'text': eval_data.full_quote.values,
    'label': [labels.index(label) for label in eval_data.target]
})


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", id2label=id2label, label2id=label2id
).to(device)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased" , max_length = MAX_LENGTH)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,max_length = MAX_LENGTH)

train_dataset = train_dataset.map(preprocess_function)
eval_dataset = eval_dataset.map(preprocess_function)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/107 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

We import the evaluate library for computing metrics while training the model

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="interview_bot",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.365621,0.242991
2,No log,2.224133,0.364486
3,No log,1.968853,0.476636
4,No log,1.629932,0.663551
5,No log,1.326915,0.757009
6,No log,1.014751,0.831776
7,No log,0.803892,0.897196
8,No log,0.623211,0.934579
9,No log,0.498690,0.943925
10,1.566200,0.407714,0.962617


TrainOutput(global_step=1080, training_loss=0.8766828201435231, metrics={'train_runtime': 7691.5161, 'train_samples_per_second': 0.278, 'train_steps_per_second': 0.14, 'total_flos': 202766382509544.0, 'train_loss': 0.8766828201435231, 'epoch': 20.0})

In [ ]:
trainer.model.save_pretrained("interview_classifier")

saving the weights of the finetuned model

##Manually testing the model's performance with new text

In [ ]:
multi_level_org_char = ['Provider characteristcs', "Health System characteristics (academics; bed capacity also)"]
multi_level_org_perspect = [ "Imaging modalities in general",
                            "Value equation: quality/cost/efficiency/patient satisfaction (Hosp leaders) High-value care (providers)",
                            "Clinical utility &  efficiency-Provider perspective",
                            "Patient/Physican interaction in LUS",
                            "Workflow (with subcoding [bolded]: access equipment, order vs encounter-based; uploading & saving images; type of equipment.  End user need to acquire and use"]

impl_sust_infra = [
    "training",
    "credientialing /quality assurance infrastructure",
    "Finanicial Impact",
]


from transformers import pipeline


text = """ I think it's variable and that's why it's so hard, school of medicine approaches it one way and specifically the department of medicine.
 There's the hospital and how they approach care, and what they value in terms of metrics and improvement efforts."""


def enter_new_text(text):
  classifier = pipeline('text-classification','interview_classifier', tokenizer = tokenizer)
  output = classifier(text)

  if output[0]['label'] in multi_level_org_char:
    output[0]['top level'] = "Multi-level organizations Characteristics-creating an environment (infrastructure) for encouraging spread "
  elif output[0]['label'] in multi_level_org_perspect:
    output[0]['top level'] = "Multi-level organizations Perspectives/Values -sharing best practices; observing results and adjusting processes accordingly"
  elif output[0]['label'] in impl_sust_infra:
    output[0]['top level'] = "Implementation and Sustainability infrastructure- facilitating use of the intervention; -ensuring adaptability of protocols that fit the multilevel context"

  print(output)



## Use the next cell to enter a new unseen text for the model to predict👇

In [ ]:
new_text = "hello"
enter_new_text(new_text)

[{'label': 'Finanicial Impact', 'score': 0.2857213616371155, 'top level': 'Implementation and Sustainability infrastructure- facilitating use of the intervention; -ensuring adaptability of protocols that fit the multilevel context'}]


#### Getting the Predictions alongside  probabilities

In [ ]:

text = 'Everybody wants something to be done, but they also want it to be done inexpensively and to save resources'

classifier = pipeline('text-classification', model='interview_classifier', tokenizer = tokenizer)

output = classifier(text, return_all_scores=True)
# Extract class names and probabilities
class_names = classifier.model.config.id2label.values()
probabilities = [i['score'] for i in output[0]]

results = sorted(zip(class_names, probabilities), key = lambda x: x[1],reverse = True)

for class_name, prob in results:
    print(f"{class_name}\n {prob}\n\n")


Finanicial Impact
 0.8623565435409546


credientialing /quality assurance infrastructure
 0.04140535369515419


Workflow (with subcoding [bolded]: access equipment, order vs encounter-based; uploading & saving images; type of equipment.  End user need to acquire and use
 0.04088936373591423


Provider characteristcs
 0.013859555125236511


Value equation: quality/cost/efficiency/patient satisfaction (Hosp leaders) High-value care (providers)
 0.007974657230079174


Clinical utility &  efficiency-Provider perspective
 0.00614489009603858


training
 0.005671518389135599


Patient/Physican interaction in LUS
 0.005476523656398058


Health System characteristics (academics; bed capacity also)
 0.00487381499260664


Imaging modalities in general
 0.004324876703321934


Clinical utility & efficiency-Provider perspective
 0.003722748253494501


Value equation: quality/cost/efficiency/patient satisfaction (Hosp leaders-background for implementation)
 0.003300220239907503




/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
